### MHI_building: this notebook should be run first when starting from the beginning (or with new data).
This notebook reads in the "Dispositions" and "Sentencing" files and checks to see which rows should be marked as having a mental health indicator (MHI). For example, if a row (case_participant_id) in the Sentencing dataset has a value of "Inpatient Mental Health Services" in the column "Commitment Type", this row would be marked as MHI = True.

In [1]:
import pandas as pd
import numpy as np
import copy
import time

disposition = pd.read_csv('Dispositions.csv', low_memory= False)
sentencing = pd.read_csv('Sentencing.csv', low_memory= False)

In [2]:
# Just getting the columns that contain MHI indicators
sent_small = sentencing[['CASE_PARTICIPANT_ID','CHARGE_DISPOSITION', 'COMMITMENT_TYPE', 
                         'CHARGE_DISPOSITION_REASON', 'SENTENCE_TYPE']]
disp_small = disposition[['CASE_PARTICIPANT_ID', 'CHARGE_DISPOSITION_REASON', 'CHARGE_DISPOSITION']]
disp_small = disp_small.rename(columns = {'CHARGE_DISPOSITION_REASON':'charge_disposition_reason_disp', 
                                          'CHARGE_DISPOSITION': 'charge_disposition_disp'}) 
sent_disp = sent_small.append(disp_small,sort=True)

In [3]:
# Checking which rows should be marked as MHI true, this will take 5-10 minutes. 

df = copy.deepcopy(sent_disp)
df.columns = map(str.lower, df.columns)

def reason_disp(series):
    return (series == 'Mental Health Graduate').any()

def disp(series):
    return series.isin(['FNG Reason Insanity',
                        'Finding Guilty But Mentally Ill',
                        'Plea of Guilty But Mentally Ill',
                        'Verdict Guilty But Mentally Ill',
                        'Sexually Dangerous Person',
                        ]).any()

def commitment_type(series):
    return series.isin(['Mental Health Probation',
                        'Inpatient Mental Health Services',
                        ]).any()

def charge_disp(series):
    return series.isin(['FNG Reason Insanity',
                        'Finding Guilty But Mentally Ill',
                        'Plea of Guilty But Mentally Ill',
                        'Verdict Guilty But Mentally Ill',
                        'Sexually Dangerous Person',
                        ]).any()

def charge_disp_reason(series):
    return (series == 'Mental Health Graduate').any()

def sentence_type(series):
    return (series == 'Inpatient Mental Health Services').any()

start_time = time.time()

grouped = df.groupby('case_participant_id').agg({'sentence_type': sentence_type, 
                                                 'charge_disposition_reason_disp': reason_disp,
                                                 'charge_disposition_disp': disp,
                                                 'commitment_type': commitment_type,
                                                 'charge_disposition': charge_disp,
                                                 'charge_disposition_reason': charge_disp_reason,
                                                 })

grouped['MHI'] = grouped.any(axis=1)

df = pd.merge(df, grouped['MHI'], left_on='case_participant_id', right_index=True)

print('total time: {}'.format(time.time() - start_time))

total time: 605.9312679767609


In [4]:
# Exporting file with just case_participant_ids where MHI = True
MHI_true = df[df['MHI']]
MHI_true_small = MHI_true[['case_participant_id', 'MHI']]
MHI_dropdup = MHI_true_small.drop_duplicates()
MHI_dropdup.to_csv('MHI_true.csv') 

MHI_dropdup

case_participant_id   MHI
186            221334022779  True
1356           252358604006  True
1601           239071986813  True
1918           248157954693  True
2027           249897124820  True
2028           249897242078  True
2198           263173188796  True
2239           268810951720  True
2269           267980413559  True
2310           263188549589  True
2315           264273068501  True
2322           267700870572  True
2939           273810596802  True
2993           271158690681  True
3683           273433612447  True
3724           275101841505  True
3757           275006276264  True
3808           275751919659  True
4214           276650584697  True
4301           276857545004  True
4449           277208732608  True
4577           277218230503  True
4633           278441817360  True
4674           277287764475  True
4830           278408398840  True
4943           265303062460  True
5217           264865455737  True
5226           264871670409  True
5342           265181231435  True
5664           269553780924  True
...                     ...   ...
633595        1027500819619  True
634711        1027830197241  True
636134        1028234268186  True
637814        1028719129868  True
640815        1029583907355  True
643737        1030314307214  True
648908        1031798793042  True
665998        1036396829597  True
666103        1036425675056  True
675489        1039113813888  True
677814        1039767996071  True
681103        1040704652689  True
686796        1042348961123  True
699525        1045875728915  True
716530        1050873966902  True
722226        1052599652362  True
727244        1054066314980  True
731934        1055338681151  True
736077        1056525918039  True
740591        1058000788714  True
744021        1059112276958  True
744637        1059298834379  True
751797        1061585950973  True
759291        1064013190834  True
780301        1071292330742  True
783820        1072550274183  True
784049        1072643142491  True
810783        1085421680925  True
812943        1087004311669  True
820459        1096325733542  True

[2439 rows x 2 columns]